In [12]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch import nn 
import copy

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, confusion_matrix, precision_score, recall_score, auc
from sklearn.model_selection import KFold
torch.manual_seed(1)    # reproducible torch:2 np:3
np.random.seed(1)

from config import BIN_config_DBPE
from models import BIN_Interaction_Flat
from stream import BIN_Data_Encoder

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Select which model to use

In [13]:
import os

model_dir = r"/home/ken/projects/MolTrans/results/davis/20220519_124222/train/model_dir"

# Load the configurations

In [14]:
config = BIN_config_DBPE()

In [15]:
config

{'batch_size': 16,
 'input_dim_drug': 23532,
 'input_dim_target': 16693,
 'train_epoch': 50,
 'max_drug_seq': 50,
 'max_protein_seq': 545,
 'emb_size': 384,
 'dropout_rate': 0.1,
 'scale_down_ratio': 0.25,
 'growth_rate': 20,
 'transition_rate': 0.5,
 'num_dense_blocks': 4,
 'kernal_dense_size': 3,
 'intermediate_size': 1536,
 'num_attention_heads': 12,
 'attention_probs_dropout_prob': 0.1,
 'hidden_dropout_prob': 0.1,
 'flat_dim': 78192}

In [16]:
model = BIN_Interaction_Flat(**config)

model = model.cuda()

# Load the model

# original saved file with DataParallel
state_dict = torch.load(os.path.join(model_dir, "max-model.ckpt"))
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)

In [17]:
model

BIN_Interaction_Flat(
  (demb): Embeddings(
    (word_embeddings): Embedding(23532, 384)
    (position_embeddings): Embedding(50, 384)
    (LayerNorm): LayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pemb): Embeddings(
    (word_embeddings): Embedding(16693, 384)
    (position_embeddings): Embedding(545, 384)
    (LayerNorm): LayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (d_encoder): Encoder_MultipleLayers(
    (layer): ModuleList(
      (0): Encoder(
        (attention): Attention(
          (self): SelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): SelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm()
            (dropo

# Protein Embedding and Fasta Embedding

In [18]:
from stream import protein2emb_encoder, drug2emb_encoder

In [19]:
params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 1,
          'drop_last': True}

In [20]:
dataFolder = './dataset/DAVIS'
df_test = pd.read_csv(dataFolder + '/test.csv')
testing_set = BIN_Data_Encoder(df_test.index.values, df_test.Label.values, df_test)
testing_generator = data.DataLoader(testing_set, **params)

In [21]:
model.eval()
y_pred = []
y_label = []
for i, (d, p, d_mask, p_mask, label) in enumerate(testing_generator):
    print(label.shape)
    score, i_map = model(d.long().cuda(), p.long().cuda(), d_mask.long().cuda(), p_mask.long().cuda())
    print(score)
    print(score.shape, i_map.shape)
    m = torch.nn.Sigmoid()
    logits = torch.squeeze(m(score))
    logits = logits.detach().cpu().numpy()
    print(logits)
    break

torch.Size([16])
tensor([[1.9992],
        [2.6540]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([2, 1]) torch.Size([16, 50, 545, 384])
[0.880718  0.9342555]


d_v_tensor = torch.from_numpy(d_v).long().cuda().unsqueeze(0)
input_mask_d_tensor = torch.from_numpy(input_mask_d).long().cuda().unsqueeze(0)
p_v_tensor = torch.from_numpy(p_v).long().cuda().unsqueeze(0)
input_mask_p_tensor = torch.from_numpy(input_mask_p).long().cuda().unsqueeze(0)
score = model.forward(d_v_tensor, 
                      p_v_tensor, 
                      input_mask_d_tensor, 
                      input_mask_p_tensor)
print(score.shape)
m = torch.nn.Sigmoid()
logits = torch.squeeze(m(score))
logits = logits.detach().cpu().numpy()
print(logits)